We start by importing all our libraries

In [1]:
import ikpy.chain
import ikpy.utils.plot as plot_utils

import numpy as np
import math

import serial

Now, we can import our robot arm model from the URDF file. The first link is the link between the desk and the base, which doesn't move, so we set it to inactive

In [2]:
my_chain = ikpy.chain.Chain.from_urdf_file("actual_arm_urdf.urdf", active_links_mask=[False, True, True, True, True, True])

And set the target position and orientation of the arm

In [3]:
target_position = [0, 0.2, 0.1]

target_orientation = [-1, 0, 0]

It's now just one call to work out the inverse kinematics for that position. Again, the first angle is of the inactive joint between the desk and the base so is always 0

In [4]:
ik = my_chain.inverse_kinematics(target_position, target_orientation, orientation_mode="Y")
print("The angles of each joints are: ", list(map(lambda r: math.degrees(r), ik.tolist())))

The angles of each joints are:  [0.0, -2.128540106467534e-08, -72.79426410142084, -120.32113697747286, 120.32113697747286, 89.9999999937032]


We can see the actual position our robot will move to. This may be different to target_position as the arm may not be physically able to reach that position

In [5]:
computed_position = my_chain.forward_kinematics(ik)
print("Computed position:", [f"{val:.2f}" for val in computed_position[:3, 3]])

Computed position: ['-0.00', '0.30', '0.08']


Now let's visualize what our arm looks

In [6]:
%matplotlib qt
import matplotlib.pyplot as plt
fig, ax = plot_utils.init_3d_figure()
fig.set_figheight(18)  
fig.set_figwidth(26)  
my_chain.plot(ik, ax, target=target_position)
plt.xlim(-0.5, 0.5)
plt.ylim(-0.5, 0.5)
ax.set_zlim(0, 0.6)
plt.ion()

Now we'll wrap up some of these calls into a couple of functions. Calling move(x,y,z) will move us to the new coordinates and update the plot.

It's worth noting here that when we call inverse_kinematics, we pass in the old position (joint angles) as initial_position so IKPY find the nearest solution to our current position.

In [7]:
def doIK():
    global ik
    old_position = ik.copy()
    ik = my_chain.inverse_kinematics(target_position, target_orientation, orientation_mode="Y", initial_position=old_position)

def updatePlot():
    ax.cla()
    my_chain.plot(ik, ax, target=target_position)
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)
    ax.set_zlim(0, 0.6)
    fig.canvas.draw_idle()
    
def move(x,y,z):
    global target_position
    target_position = [x,y,z]
    doIK()
    updatePlot()

    #sendCommand(ik[1].item(),ik[2].item(),ik[3].item(),ik[4].item(),ik[5].item(),ik[6].item(),1)

We should now be able to move our visualized arm with a call to move(x,y,z)

In [19]:
move(0.3, 0.2, 0.5)